In [1]:
import os, requests, urllib, zipfile
from bs4 import BeautifulSoup as bs
import xml.etree.ElementTree as ET
from datetime import datetime as date

# root domain of German parliament website
root = 'https://www.bundestag.de'
# ajax request url
asin = '/ajax/filterlist/de/services/opendata/488214-488214?limit=10&noFilterSet=true'

# HTTP request
page = requests.get(root + asin)


In [ ]:

# assign BeautifulSoup object on response content
soup = bs(page.content)

# TODO: clean the data folder first
directory = os.fsencode('./data/xml/')


# loop all anchor with this class
for a in soup.find_all(attrs={'class':'bt-link-dokument'}):
    # retrieve plenary zip file from hrefs
    zipcall, _ = urllib.request.urlretrieve(root + a['href'])
    # extract all xml files of this legislature into raw folder
    with zipfile.ZipFile(zipcall, "r") as f:
        f.extractall(xmldir)



In [22]:
import xml.etree.ElementTree as ET
from datetime import datetime as date

# data paths (TODO: move back to top)
data_root = './data/'
xmldir = data_root + 'xml/'
txtdir = data_root + 'txt/'

# PDFs headers to be removed in some of the xml files
headingLines = ['Deutscher Bundestag –', '(A)', '(B)']

# iterate over all the .xml files in the folder
for file in os.listdir(os.fsencode(xmldir)):
    # decode the name of file
    filename = os.fsdecode(file)
    # work on just .xmls and remove everything else
    if filename.endswith('.xml'):
        # element tree XML parsing : get root
        root = ET.parse(xmldir + filename).getroot()
        # TEXT tag first to acquire the plenary body
        for child in root.iter('TEXT'):
            with open('./data/txt/temp.txt', 'w') as tempfile:
                tempfile.write(child.text)
        # DATUM as the date of the plenary for file renaming
        for child in root.iter('DATUM'):
            plenaryDate = date.strptime(child.text, '%d.%m.%Y').strftime('%Y-%m-%d')
            with open(txtdir + 'temp.txt') as tempfile, open(txtdir + plenaryDate + '.txt', 'w') as textfile:
                for line in tempfile:
                    if not any(header in line for header in headingLines):
                        textfile.write(line)
    else: 
        os.remove(xmldir + filename)

    os.remove(txtdir + 'temp.txt')


In [21]:

# We can do it later when we work on the single lines

import dehyphen

scorer = dehyphen.FlairScorer(lang="de")

with open('./data/txt/temp.txt', 'r') as file:
    special_format = dehyphen.text_to_format(file.read())
    cleaned_text = scorer.dehyphen(special_format)